In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
# Load Iris dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target   # Features and labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [3]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)   # 4 features -> hidden layer
        self.fc2 = nn.Linear(16, 12)
        self.fc3 = nn.Linear(12, 3)   # 3 classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = IrisNet()


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)



In [5]:
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.7252
Epoch [20/100], Loss: 0.4054
Epoch [30/100], Loss: 0.3065
Epoch [40/100], Loss: 0.1959
Epoch [50/100], Loss: 0.0995
Epoch [60/100], Loss: 0.0642
Epoch [70/100], Loss: 0.0520
Epoch [80/100], Loss: 0.0461
Epoch [90/100], Loss: 0.0429
Epoch [100/100], Loss: 0.0414


In [6]:
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)

print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 100.00%


In [9]:

# STEP 1: Import Libraries

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


# STEP 2: Load and Preprocess Data

iris = datasets.load_iris()
X, y = iris.data, iris.target   # 150 samples, 4 features, 3 classes

# Split into Train, Validation, Test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# Standardize (important for NN training!)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


# STEP 3: Define Model (with Dropout)

class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # 30% neurons off during training

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = IrisNet()


# STEP 4: Loss & Optimizer (with Regularization)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)  # L2 regularization


# STEP 5: Training with Early Stopping
epochs = 200
best_val_loss = float("inf")
patience, patience_counter = 20, 0  # Early stopping patience

for epoch in range(epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()  # Save best model
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"⏹ Early stopping at epoch {epoch+1}")
        break

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# Load the best model
model.load_state_dict(best_model_state)

# STEP 6: Final Evaluation

model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)

print(f"Final Test Accuracy: {accuracy*100:.2f}%")


Epoch 10, Train Loss: 0.8535, Val Loss: 0.7457
Epoch 20, Train Loss: 0.5443, Val Loss: 0.4481
Epoch 30, Train Loss: 0.4618, Val Loss: 0.2981
Epoch 40, Train Loss: 0.3104, Val Loss: 0.2388
Epoch 50, Train Loss: 0.3427, Val Loss: 0.2064
Epoch 60, Train Loss: 0.2101, Val Loss: 0.1624
Epoch 70, Train Loss: 0.1467, Val Loss: 0.1537
Epoch 80, Train Loss: 0.1689, Val Loss: 0.1364
Epoch 90, Train Loss: 0.1518, Val Loss: 0.1736
Epoch 100, Train Loss: 0.1734, Val Loss: 0.1436
⏹ Early stopping at epoch 102
Final Test Accuracy: 96.67%
